# TP1 de Machine Learning

Cours de François Boussion

In [10]:
from pandas import read_csv 

with open('housing.csv', 'r') as f:
    df = read_csv(f, header=0, sep=',', index_col=False)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


*Combien y a-t-il de variables ? Quel est leur type ? Y a-t-il des données manquantes ?*

Il y a 10 variables. La majorité sont des nombres flottants, mais `ocean_proximity` est un "object" (en réalité, une chaîne de caractères).

In [22]:
def get_missing(df):
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    return missing

print(get_missing(df))

total_bedrooms    207
dtype: int64


Il y a 207 valeurs manquantes pour la variable total_bedrooms.